In [ ]:
import os
import config
from dataloader.loader import Loader
from preprocessing.utils import Preprocess, remove_empty_docs
from dataloader.embeddings import GloVe
from model.cnn_document_model import DocumentModel, TrainingParameters
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

In [ ]:
# 네이버 영화 리뷰 데이터를 로드하여 데이터프레임으로 변환한다
train_df = Loader.load_amazon_reviews('train')
print(f'train_df.shape : {train_df.shape}')

test_df = Loader.load_amazon_reviews('test')
print(f'test_df.shape : {test_df.shape}')

In [ ]:
# 학습셋에서 랜덤으로 2만개만 추출하여 feature 추출에 사용한다
dataset = train_df.sample(n=20000, random_state=6)
dataset.sentiment.value_counts()

In [ ]:
train_df.head()

In [ ]:
# 추출한 20만개 데이터 샘플에서 review, sentiment 칼럼 값들 추출
corpus = dataset['review'].values
target = dataset['sentiment'].values
print(f'corpus.shape : {corpus.shape}')
print(f'target.shape : {target.shape}')

# 유효하지 않은 값 제거 (비어있거나 길이가 30 이하인 경우 제거)
corpus, target = remove_empty_docs(corpus, target)
print('=== after remove_empty_docs ===')
print(f'corpus size : {len(corpus)}')
print(f'target size : {len(target)}')

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

In [ ]:
print(f'corpus_to_seq size : {len(corpus_to_seq)}')
print(f'corpus_to_seq[0] size : {len(corpus_to_seq[0])}')
print(f'corpus_to_seq[0] :')
print(corpus_to_seq[0])

In [ ]:
# 인덱싱되기 전 원본 문서
corpus[1]

In [ ]:
holdout_corpus = test_df['review'].values
holdout_target = test_df['sentiment'].values
print(f'holdout_corpus.shape : {holdout_corpus.shape}')
print(f'holdout_target.shape : {holdout_target.shape}')

holdout_corpus, holdout_target = remove_empty_docs(holdout_corpus, holdout_target)
print('=== after remove_empty_docs ===')
print(f'holdout_corpus size : {len(holdout_corpus)}')
print(f'holdout_target size : {len(holdout_target)}')

In [ ]:
# 테스트셋을 인덱스 시퀀스로 변환 (위에서 생성한 인덱스 사전 그대로 사용)
holdout_corpus_to_seq = preprocessor.transform(holdout_corpus)

In [ ]:
print(f'holdout_corpus_to_seq size : {len(holdout_corpus_to_seq)}')
print(f'holdout_corpus_to_seq[0] size : {len(holdout_corpus_to_seq[0])}')
print(f'holdout_corpus_to_seq[0] :')
print(holdout_corpus_to_seq[0])

In [ ]:
# 인덱싱된 텍스트 데이터를 GloVe로 임베딩 초기화.
# glove.6B.50d.txt에 없는 단어는 OOV..txt에 write한다
# word_index는 {'expensive': 2, 'junk': 3, 'this': 4, ...} 형태의 인덱싱 사전
glove = GloVe(50)
initial_embeddings = glove.get_embedding(preprocessor.word_index)  

In [ ]:
# 인덱스 사전의 단어 수
len(preprocessor.word_index)

In [ ]:
# GloVe로 임베딩 초기화된 행렬. 벡터 개수는 word_index 인덱스 사전의 단어 + 2, 차원 수는 50이다
initial_embeddings.shape

In [ ]:
# CNN 기반 문서 분류 모델 인스턴스 생성. 위에서 GloVe로 만든 임베딩 행렬을 임베딩 초깃값으로 사용한다
naver_review_model = DocumentModel(vocab_size=preprocessor.get_vocab_size(),
                                    word_index=preprocessor.word_index,
                                    num_sentences=Preprocess.NUM_SENTENCES,
                                    embedding_weights=initial_embeddings,
                                    conv_activation='tanh',
                                    hidden_dims=64,
                                    input_dropout=0.40,
                                    hidden_gaussian_noise_sd=0.5)

In [ ]:
# 학습된 모델을 저장할 디렉토리 생성
if not os.path.exists(os.path.join(config.MODEL_DIR, 'amazonreviews')):
    os.makedirs(os.path.join(config.MODEL_DIR, 'amazonreviews'))

# 학습 파라미터 저장 클래스
train_params = TrainingParameters('model_with_tanh_activation', 
                                  model_file_path = config.MODEL_DIR + '/amazonreviews/naver_model.hdf5',
                                  model_hyper_parameters = config.MODEL_DIR + '/amazonreviews/naver_model.json',
                                  model_train_parameters = config.MODEL_DIR + '/amazonreviews/naver_model_meta.json',
                                  num_epochs=1000)

# 모델 컴파일
naver_review_model.get_classification_model().compile(loss="binary_crossentropy", 
                                                       optimizer=train_params.optimizer,
                                                       metrics=["accuracy"])

# callback (1) - 자동저장 체크포인트
checkpointer = ModelCheckpoint(filepath=train_params.model_file_path,
                               verbose=1,
                               save_best_only=True,
                               save_weights_only=True)

# callback (2) - 조기종료
early_stop = EarlyStopping(patience=2)

# 모델에 입력할 학습데이터, 테스트데이터 (인덱스 값들의 시퀀스로 변환된 값)
x_train = np.array(corpus_to_seq)
y_train = np.array(target)
x_test = np.array(holdout_corpus_to_seq)
y_test = np.array(holdout_target)
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_test.shape : {x_test.shape}')
print(f'y_test.shape : {y_test.shape}')

# 모델 훈련 시작
history = naver_review_model.get_classification_model().fit(x_train,
                                                   y_train, 
                                                   batch_size=train_params.batch_size, 
                                                   epochs=train_params.num_epochs,  # 35 epochs
                                                   verbose=2,
                                                   validation_split=train_params.validation_split, # 5%
                                                   callbacks=[checkpointer])

# 모델 저장
naver_review_model._save_model(train_params.model_hyper_parameters)

In [ ]:
history.history['accuracy']

In [ ]:
history.history['loss']

In [ ]:
history.history['val_accuracy']

In [ ]:
history.history['val_loss']

In [ ]:
# 모델 평가 - 테스트 데이터셋으로 수행
naver_review_model.get_classification_model().evaluate(x_test,
                                                        y_test, 
                                                        train_params.batch_size*10,
                                                        verbose=2)

In [ ]:
learned_embeddings = naver_review_model.get_classification_model().get_layer('imdb_embedding').get_weights()[0]

embd_change = {}
for word, i in preprocessor.word_index.items():
    # Frobenium norm (Euclidean norm) 계
    embd_change[word] = np.linalg.norm(initial_embeddings[i]-learned_embeddings[i])
embd_change = sorted(embd_change.items(), key=lambda x: x[1], reverse=True)
embd_change[0:100]